In [1]:
import os
from google.colab import drive

# --- Mount drive and basic config ---
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/protein-multimodal')

Mounted at /content/drive


In [2]:
PROJECT_ROOT = "/content/drive/MyDrive/protein-multimodal"
DATA_DIR = f"{PROJECT_ROOT}/data"
RAW_DIR = f"{DATA_DIR}/raw"
PROCESSED_DIR = f"{DATA_DIR}/processed"
EMBEDDINGS_DIR = f"{DATA_DIR}/embeddings"
MODELS_DIR = f"{PROJECT_ROOT}/models"
NEW_DATA_DIR = f"{DATA_DIR}/new_proteins"

os.makedirs(NEW_DATA_DIR, exist_ok=True)


In [3]:
# Install dependencies
!pip install -q biopython biotite transformers datasets accelerate peft
!pip install -q fair-esm umap-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.8 MB/s eta 0:00:00


In [4]:
import json
import re
import time
import requests
import torch
import esm
import numpy as np
import pickle
from tqdm.notebook import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import umap

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n🖥️  Device: {device}")
if device.type == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")



🖥️  Device: cuda
   GPU: Tesla T4
   Memory: 15.83 GB


In [6]:
"""### **Load Existing Accessions to Avoid Duplicates**"""

def load_existing_accessions():
    """Load all existing protein accessions from raw data"""
    existing_accessions = set()

    raw_file = f"{RAW_DIR}/proteins_raw.json"
    if os.path.exists(raw_file):
        with open(raw_file, "r") as f:
            existing_data = json.load(f)
            existing_accessions.update([p["accession"] for p in existing_data])
        print(f"✓ Loaded {len(existing_accessions)} existing accessions from raw data")
    else:
        print("⚠️  No existing raw data file found. Will fetch from scratch.")

    return existing_accessions

existing_accessions = load_existing_accessions()
print(f"Total existing proteins to exclude: {len(existing_accessions)}")

✓ Loaded 413 existing accessions from raw data
Total existing proteins to exclude: 413


In [7]:
import json

# Check raw data
with open('/content/drive/MyDrive/protein-multimodal/data/raw/proteins_raw.json', 'r') as f:
    raw_data = json.load(f)
    print(f"Raw data: {len(raw_data)} proteins")

# Check processed splits
with open('/content/drive/MyDrive/protein-multimodal/data/processed/train.json', 'r') as f:
    train = json.load(f)
with open('/content/drive/MyDrive/protein-multimodal/data/processed/val.json', 'r') as f:
    val = json.load(f)
with open('/content/drive/MyDrive/protein-multimodal/data/processed/test.json', 'r') as f:
    test = json.load(f)

print(f"Train: {len(train)} proteins")
print(f"Val: {len(val)} proteins")
print(f"Test: {len(test)} proteins")
print(f"Total processed: {len(train) + len(val) + len(test)} proteins")

Raw data: 10000 proteins
Train: 8000 proteins
Val: 1000 proteins
Test: 1000 proteins
Total processed: 10000 proteins


In [8]:
# File paths
RAW_FILE = '/content/drive/MyDrive/protein-multimodal/data/raw/proteins_raw.json'
TRAIN_FILE = '/content/drive/MyDrive/protein-multimodal/data/processed/train.json'
VAL_FILE = '/content/drive/MyDrive/protein-multimodal/data/processed/val.json'
TEST_FILE = '/content/drive/MyDrive/protein-multimodal/data/processed/test.json'


In [9]:
# ============================================================================
# LOAD ALL DATASETS
# ============================================================================

print("\n📂 Loading datasets...")

with open(RAW_FILE, 'r') as f:
    raw_data = json.load(f)

with open(TRAIN_FILE, 'r') as f:
    train_data = json.load(f)

with open(VAL_FILE, 'r') as f:
    val_data = json.load(f)

with open(TEST_FILE, 'r') as f:
    test_data = json.load(f)

print(f"✓ Raw data: {len(raw_data)} proteins")
print(f"✓ Train: {len(train_data)} proteins")
print(f"✓ Val: {len(val_data)} proteins")
print(f"✓ Test: {len(test_data)} proteins")


📂 Loading datasets...
✓ Raw data: 10000 proteins
✓ Train: 8000 proteins
✓ Val: 1000 proteins
✓ Test: 1000 proteins


In [10]:
# ============================================================================
# EXTRACT ACCESSIONS
# ============================================================================

print("\n🔍 Extracting accession IDs...")

raw_accessions = set([p['accession'] for p in raw_data])
train_accessions = set([p['accession'] for p in train_data])
val_accessions = set([p['accession'] for p in val_data])
test_accessions = set([p['accession'] for p in test_data])

print(f"✓ Unique accessions - Raw: {len(raw_accessions)}")
print(f"✓ Unique accessions - Train: {len(train_accessions)}")
print(f"✓ Unique accessions - Val: {len(val_accessions)}")
print(f"✓ Unique accessions - Test: {len(test_accessions)}")


🔍 Extracting accession IDs...
✓ Unique accessions - Raw: 413
✓ Unique accessions - Train: 413
✓ Unique accessions - Val: 375
✓ Unique accessions - Test: 382


In [12]:
# ============================================================================
# CHECK FOR DUPLICATES WITHIN EACH SPLIT
# ============================================================================
from collections import Counter
print("\n" + "=" * 80)
print("1. CHECKING FOR DUPLICATES WITHIN EACH SPLIT")
print("=" * 80)

def check_duplicates(data, name):
    accessions = [p['accession'] for p in data]
    counter = Counter(accessions)
    duplicates = {acc: count for acc, count in counter.items() if count > 1}

    if duplicates:
        print(f"\n❌ {name}: Found {len(duplicates)} duplicate accessions!")
        for acc, count in list(duplicates.items())[:5]:
            print(f"   - {acc}: appears {count} times")
        if len(duplicates) > 5:
            print(f"   ... and {len(duplicates) - 5} more")
        return False
    else:
        print(f"✅ {name}: No duplicates found")
        return True

raw_ok = check_duplicates(raw_data, "Raw data")
train_ok = check_duplicates(train_data, "Train split")
val_ok = check_duplicates(val_data, "Val split")
test_ok = check_duplicates(test_data, "Test split")


1. CHECKING FOR DUPLICATES WITHIN EACH SPLIT

❌ Raw data: Found 413 duplicate accessions!
   - A0A1B0GTW7: appears 25 times
   - A0PK11: appears 25 times
   - A1A4S6: appears 25 times
   - A1A519: appears 25 times
   - A1L190: appears 25 times
   ... and 408 more

❌ Train split: Found 413 duplicate accessions!
   - O60235: appears 18 times
   - P01732: appears 21 times
   - A7MD48: appears 21 times
   - O96000: appears 16 times
   - O14503: appears 20 times
   ... and 408 more

❌ Val split: Found 289 duplicate accessions!
   - O15116: appears 2 times
   - O75815: appears 5 times
   - O00478: appears 2 times
   - O95977: appears 2 times
   - P09544: appears 5 times
   ... and 284 more

❌ Test split: Found 295 duplicate accessions!
   - O43529: appears 3 times
   - O95793: appears 3 times
   - O14843: appears 4 times
   - P09912: appears 4 times
   - O14967: appears 6 times
   ... and 290 more


In [13]:
# ============================================================================
# CHECK FOR OVERLAPS BETWEEN SPLITS (DATA LEAK)
# ============================================================================

print("\n" + "=" * 80)
print("2. CHECKING FOR DATA LEAKS BETWEEN SPLITS")
print("=" * 80)

def check_overlap(set1, set2, name1, name2):
    overlap = set1 & set2
    if overlap:
        print(f"\n❌ DATA LEAK DETECTED: {name1} ∩ {name2}")
        print(f"   Found {len(overlap)} overlapping proteins!")
        print(f"   Sample overlapping accessions: {list(overlap)[:5]}")
        return overlap
    else:
        print(f"✅ No overlap: {name1} ∩ {name2}")
        return set()

# Check all pairwise overlaps
train_val_overlap = check_overlap(train_accessions, val_accessions, "Train", "Val")
train_test_overlap = check_overlap(train_accessions, test_accessions, "Train", "Test")
val_test_overlap = check_overlap(val_accessions, test_accessions, "Val", "Test")


2. CHECKING FOR DATA LEAKS BETWEEN SPLITS

❌ DATA LEAK DETECTED: Train ∩ Val
   Found 375 overlapping proteins!
   Sample overlapping accessions: ['O75781', 'A5D8V7', 'P06028', 'O43914', 'A1A4S6']

❌ DATA LEAK DETECTED: Train ∩ Test
   Found 382 overlapping proteins!
   Sample overlapping accessions: ['O75781', 'P06028', 'O43914', 'A1A4S6', 'O95479']

❌ DATA LEAK DETECTED: Val ∩ Test
   Found 344 overlapping proteins!
   Sample overlapping accessions: ['O75781', 'P06028', 'O43914', 'A1A4S6', 'O95479']


In [14]:
# ============================================================================
# CHECK IF ALL PROCESSED DATA COMES FROM RAW DATA
# ============================================================================

print("\n" + "=" * 80)
print("3. CHECKING DATA CONSISTENCY")
print("=" * 80)

all_processed = train_accessions | val_accessions | test_accessions

# Check if all processed proteins are in raw data
not_in_raw = all_processed - raw_accessions
if not_in_raw:
    print(f"\n⚠️  Warning: {len(not_in_raw)} processed proteins NOT found in raw data!")
    print(f"   Sample: {list(not_in_raw)[:5]}")
else:
    print("✅ All processed proteins exist in raw data")

# Check if raw data was fully used
not_processed = raw_accessions - all_processed
if not_processed:
    print(f"\n⚠️  Warning: {len(not_processed)} raw proteins were NOT processed!")
    print(f"   These were likely filtered out during quality control")
    print(f"   Sample: {list(not_processed)[:5]}")
else:
    print("✅ All raw proteins were processed")


3. CHECKING DATA CONSISTENCY
✅ All processed proteins exist in raw data
✅ All raw proteins were processed


In [15]:
# ============================================================================
# CHECK SPLIT RATIOS
# ============================================================================

print("\n" + "=" * 80)
print("4. SPLIT RATIO ANALYSIS")
print("=" * 80)

total_processed = len(all_processed)
train_ratio = len(train_accessions) / total_processed * 100
val_ratio = len(val_accessions) / total_processed * 100
test_ratio = len(test_accessions) / total_processed * 100

print(f"\nSplit ratios (based on {total_processed} total proteins):")
print(f"  Train: {len(train_accessions):>5} proteins ({train_ratio:>5.2f}%)")
print(f"  Val:   {len(val_accessions):>5} proteins ({val_ratio:>5.2f}%)")
print(f"  Test:  {len(test_accessions):>5} proteins ({test_ratio:>5.2f}%)")
print(f"  Total: {total_processed:>5} proteins (100.00%)")

expected_train = 0.80
expected_val = 0.10
expected_test = 0.10

train_diff = abs(train_ratio/100 - expected_train)
val_diff = abs(val_ratio/100 - expected_val)
test_diff = abs(test_ratio/100 - expected_test)

if train_diff < 0.01 and val_diff < 0.01 and test_diff < 0.01:
    print("\n✅ Split ratios match expected 80/10/10 distribution")
else:
    print(f"\n⚠️  Split ratios deviate from expected 80/10/10:")
    print(f"   Train: {abs(train_ratio - 80):.2f}% difference")
    print(f"   Val: {abs(val_ratio - 10):.2f}% difference")
    print(f"   Test: {abs(test_ratio - 10):.2f}% difference")


4. SPLIT RATIO ANALYSIS

Split ratios (based on 413 total proteins):
  Train:   413 proteins (100.00%)
  Val:     375 proteins (90.80%)
  Test:    382 proteins (92.49%)
  Total:   413 proteins (100.00%)

⚠️  Split ratios deviate from expected 80/10/10:
   Train: 20.00% difference
   Val: 80.80% difference
   Test: 82.49% difference


In [16]:
# ============================================================================
# CHECK SEQUENCE AND FUNCTION CONSISTENCY
# ============================================================================

print("\n" + "=" * 80)
print("5. CHECKING SEQUENCE AND FUNCTION CONSISTENCY")
print("=" * 80)

# Create lookup dictionaries from raw data
raw_lookup = {p['accession']: p for p in raw_data}

def check_data_consistency(processed_data, split_name):
    """Check if sequences and functions match raw data"""
    mismatches = []

    for protein in processed_data:
        acc = protein['accession']
        if acc not in raw_lookup:
            continue

        raw_protein = raw_lookup[acc]

        # Check sequence
        if protein['sequence'] != raw_protein['sequence']:
            mismatches.append({
                'accession': acc,
                'type': 'sequence',
                'split': split_name
            })

        # Check function
        if protein['function'] != raw_protein['function']:
            mismatches.append({
                'accession': acc,
                'type': 'function',
                'split': split_name
            })

    return mismatches

print("\nChecking data consistency across splits...")
train_mismatches = check_data_consistency(train_data, "Train")
val_mismatches = check_data_consistency(val_data, "Val")
test_mismatches = check_data_consistency(test_data, "Test")

all_mismatches = train_mismatches + val_mismatches + test_mismatches

if all_mismatches:
    print(f"\n❌ Found {len(all_mismatches)} data inconsistencies!")
    df = pd.DataFrame(all_mismatches)
    print("\nBreakdown by type:")
    print(df.groupby(['split', 'type']).size())
else:
    print("✅ All sequences and functions match raw data")


5. CHECKING SEQUENCE AND FUNCTION CONSISTENCY

Checking data consistency across splits...
✅ All sequences and functions match raw data


In [17]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("FINAL SUMMARY")
print("=" * 80)

all_checks_passed = True

if not (raw_ok and train_ok and val_ok and test_ok):
    print("❌ FAILED: Duplicate accessions found within splits")
    all_checks_passed = False

if train_val_overlap or train_test_overlap or val_test_overlap:
    print("❌ FAILED: Data leak detected between splits!")
    print(f"   Total leaked proteins: {len(train_val_overlap | train_test_overlap | val_test_overlap)}")
    all_checks_passed = False
else:
    print("✅ PASSED: No data leaks between splits")

if not_in_raw:
    print(f"⚠️  WARNING: {len(not_in_raw)} processed proteins not in raw data")
    all_checks_passed = False

if all_mismatches:
    print(f"❌ FAILED: {len(all_mismatches)} data inconsistencies detected")
    all_checks_passed = False

if all_checks_passed:
    print("\n" + "🎉" * 40)
    print("ALL CHECKS PASSED! Your dataset is clean and properly split.")
    print("🎉" * 40)
else:
    print("\n⚠️  Some issues were detected. Please review the output above.")



FINAL SUMMARY
❌ FAILED: Duplicate accessions found within splits
❌ FAILED: Data leak detected between splits!
   Total leaked proteins: 413

⚠️  Some issues were detected. Please review the output above.


In [18]:
# ============================================================================
# SAVE DETAILED REPORT
# ============================================================================

report = {
    "summary": {
        "raw_proteins": len(raw_data),
        "train_proteins": len(train_data),
        "val_proteins": len(val_data),
        "test_proteins": len(test_data),
        "total_processed": total_processed,
        "all_checks_passed": all_checks_passed
    },
    "duplicates": {
        "raw_has_duplicates": not raw_ok,
        "train_has_duplicates": not train_ok,
        "val_has_duplicates": not val_ok,
        "test_has_duplicates": not test_ok
    },
    "data_leaks": {
        "train_val_overlap": len(train_val_overlap),
        "train_test_overlap": len(train_test_overlap),
        "val_test_overlap": len(val_test_overlap),
        "overlapping_accessions": list(train_val_overlap | train_test_overlap | val_test_overlap)
    },
    "consistency": {
        "proteins_not_in_raw": len(not_in_raw),
        "raw_proteins_not_processed": len(not_processed),
        "data_mismatches": len(all_mismatches)
    },
    "split_ratios": {
        "train": round(train_ratio, 2),
        "val": round(val_ratio, 2),
        "test": round(test_ratio, 2)
    }
}

report_path = '/content/drive/MyDrive/protein-multimodal/data/data_leak_report.json'
with open(report_path, 'w') as f:
    json.dump(report, f, indent=2)

print(f"\n📄 Detailed report saved to: {report_path}")


📄 Detailed report saved to: /content/drive/MyDrive/protein-multimodal/data/data_leak_report.json
